<a href="https://colab.research.google.com/github/yash2998chhabria/Rnn-Nlp/blob/master/Django_Runnnable_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Django-Setting run

In [5]:
import spacy
import torch
import torchtext
import re
import torch.nn as nn

Only For Drive


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#/content/drive/MyDrive/STATE DICT OF SCRAPPED RNN DATASET/deprnnscrapped_state_dic

In [9]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

def tokenizer(s): 
    return [w.text.lower() for w in nlp(tweet_clean(s))]

In [10]:
TEXT = torchtext.data.Field(tokenize = tokenizer)

LABEL = torchtext.data.LabelField(dtype = torch.float)

In [11]:
def tweet_clean(text):
    
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text) 
    text = re.sub(r'https?:/\/\S+', ' ', text) 
    
    return text.strip()

In [12]:
datafields = [('SentimentText', TEXT),('Sentiment', LABEL)]

In [13]:
trn, tst = torchtext.data.TabularDataset.splits(path = '/content/drive/MyDrive/depressionrnn/', 
                                                train = 'train_tweets.csv',
                                                test = 'test_tweets.csv',    
                                                format = 'csv',
                                                skip_header = True,
                                                fields = datafields)

In [14]:
TEXT.build_vocab(trn, max_size=25000,
                 vectors="glove.6B.100d",
                 unk_init=torch.Tensor.normal_)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                          
100%|█████████▉| 398619/400000 [00:22<00:00, 17919.29it/s]

In [8]:

class RNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, 
                 output_dim, n_layers, bidirectional, dropout):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers = n_layers, 
                           bidirectional = bidirectional, dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        
        self.dropout = nn.Dropout(dropout)

        
    def forward(self, text):
        
        embedded = self.dropout(self.embedding(text))
        
        output, hidden = self.rnn(embedded)
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
       
        return self.fc(hidden.squeeze(0))

In [15]:
input_dim = len(TEXT.vocab)

embedding_dim = 100

hidden_dim = 20
output_dim = 1

n_layers = 2
bidirectional = True

dropout = 0.5

In [16]:
new_model = RNN(input_dim, 
            embedding_dim, 
            hidden_dim, 
            output_dim, 
            n_layers, 
            bidirectional, 
            dropout)

In [18]:
new_model.load_state_dict(torch.load('/content/drive/MyDrive/STATE DICT OF SCRAPPED RNN DATASET/deprnnscrapped_state_dic'))

<All keys matched successfully>

#Django views

In [34]:
sentence = 'Im happy.'

In [35]:
tokenized = [tok.text for tok in nlp.tokenizer(sentence)]

In [36]:
indexed = [TEXT.vocab.stoi[t] for t in tokenized]

In [37]:
tensor = torch.LongTensor(indexed)

In [38]:
tensor = tensor.unsqueeze(1)

In [39]:
prediction = torch.sigmoid(new_model(tensor))

In [40]:
if prediction.item() > 0.5:
  print("Depression Symptoms Detected")
else:
  print("No Symptoms Detected")  

prediction.item()

No Symptoms Detected


0.44792094826698303